# Chainerによる実践深層学習

In [1]:
import chainer
import numpy as np
import copy
import pickle

### Numpyで最低限知っておくべきこと．

In [2]:
# 配列の形を知りたい時はshape， データ数を知りたい時はsize
a = np.arange(60).reshape(10, 6)
nrow, ncol = a.shape
print(nrow, ncol) 
print(a.size)

10 6
60


In [3]:
# 0や1以外での初期化
print(np.empty(10))

[ -2.31584178e+077  -4.32912595e-311   2.29425253e-314   2.29429626e-314
   2.29430218e-314   2.29423600e-314   2.23047865e-314   0.00000000e+000
   2.21638048e-314   1.27319747e-310]


In [4]:
# 乱数の生成． 通常は一様分布uniformと正規分布normalを知っていれば問題ない．
print(np.random.uniform(0, 1, 3)) # 区間(0,1)の一様分布に従う乱数を3つ生成
print(np.random.normal(1.5, 2.0, 3)) # 平均1.5, 標準偏差2,0の正規分布に従う乱数を3個生成

[ 0.95218803  0.36405546  0.44882674]
[-0.02471806  1.06972271  0.6789234 ]


In [17]:
# 要素をシャッフルした配列を生成． shuffleは配列を破壊的に並び替えるので， 通常はpermutatonを使うのが安全
x = np.arange(6)

# permutationは値のコピー
y = np.random.permutation(x)
print(y)
print(x, end="\n\n")

# shuffleはin-place(破壊的、ポインタのコピー)
z = np.random.shuffle(x)
print(z)
print(x)

[3 2 0 5 4 1]
[0 1 2 3 4 5]

None
[1 2 5 4 3 0]


In [18]:
# 単位行列の生成
np.identity(5)

array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [19]:
#  配列の結合
a = np.arange(6).reshape(2, 3)
b = np.arange(6, 12).reshape(2, 3)
print(np.hstack([a,b]), end="\n\n") # horizontal
print(np.vstack([a,b])) # virtical

[[ 0  1  2  6  7  8]
 [ 3  4  5  9 10 11]]

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]


In [20]:
# スライス
a = np.arange(30).reshape(5, 6)
print(a, end="\n\n")
print(a[[0, 2,4], 2])

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]]

[ 2 14 26]


In [21]:
# 置換
a = np.arange(30).reshape(5, 6)
a[a%2 == 0] = -1
print(a)

[[-1  1 -1  3 -1  5]
 [-1  7 -1  9 -1 11]
 [-1 13 -1 15 -1 17]
 [-1 19 -1 21 -1 23]
 [-1 25 -1 27 -1 29]]


### 値渡しと参照渡し

値渡し；変数の値だけをコピーする渡し方．「実体のコピー」  
参照渡し：変数を共有するような渡し方．「ポインタのコピー」  

    
Pythonでは，変数や関数に値を渡す場合すべて参照渡し．Immutable(変更不可)な型のみ，値が変更されたときに新たなオブジェクトを生成する(元のオブジェクトは変更されない)．
Mutableな型で値渡しをしたい場合はcopy()を利用．

- 変更不可（Immutable）な型  
    - int, float, str, tuple, bytes, frozenset 等  
- 変更可能（Mutable）な型  
    - list, dict, set, bytearray 等  

In [28]:
#  値渡しと参照渡し
def foo(a):
    print("a = %d, id(a) = %d"%(a, id(a))) # bと同じ
    a += 1
    print("a = %d, id(a) = %d"%(a, id(a))) # 新たな領域が確保される

b = 0
print("b = %d, id(b) = %d"%(b, id(b))) # bの領域が確保される
foo(b)
print("b = %d, id(b) = %d"%(b, id(b))) # 値は変更されていない

b = 0, id(b) = 4482562992
a = 0, id(a) = 4482562992
a = 1, id(a) = 4482563024
b = 0, id(b) = 4482562992


In [31]:
# コピー
a = np.arange(6).reshape(2,3)
print(a, end="\n\n")

b = a # ポインタのコピー
b[0] = 0
print(a, end="\n\n") # 元の配列も変更される

c = a.copy() # 実体のコピー
c[0] = 1
print(a) # 元の配列は変更されない．

[[0 1 2]
 [3 4 5]]

[[0 0 0]
 [3 4 5]]

[[0 0 0]
 [3 4 5]]


In [32]:
# 行列の積(配列が1次元(ベクトル)の場合は内積)
a = np.arange(4)
b = np.arange(4, 8)
print(a.dot(b), end="\n\n") # 内積

a = np.arange(6).reshape(2,3)
b = np.arange(6).reshape(3,2)
print(a.dot(b))

38

[[10 13]
 [28 40]]


In [33]:
 # 行列の演算は逆行列， 転置行列， 行列式，　固有値が重要．
a = np.array([[0, 6, 3], [-2, 7, 2], [0, 0, 3]])
print(a.T, end="\n\n") # 転置
print(np.linalg.det(a), end="\n\n") # 行列式
print(np.linalg.inv(a), end="\n\n") # 逆行列
la, v = np.linalg.eig(a)
print(la, end="\n\n") # 固有値
print(v) # 固有ベクトル

[[ 0 -2  0]
 [ 6  7  0]
 [ 3  2  3]]

36.0

[[ 0.58333333 -0.5        -0.25      ]
 [ 0.16666667  0.         -0.16666667]
 [ 0.          0.          0.33333333]]

[ 3.  4.  3.]

[[-0.89442719 -0.83205029  0.43643578]
 [-0.4472136  -0.5547002  -0.21821789]
 [ 0.          0.          0.87287156]]


In [35]:
# pickleを利用した配列の保存と呼び出し． pickleは配列に鍵らずどのようなオブジェクトでも保存とその読み出しが汎用的にできる．
a = np.random.randn(10).reshape(2, 5)
print(a, end="\n\n")

# 書き出し
f = open('a.pickle', 'wb') # a.pickleに書き出される． write byte． wだけだとエラー．
pickle.dump(a, f) # dump：放出する．
f.close()

# 読み込み
f = open("a.pickle", "rb") # read byte
a = pickle.load(f)
f.close()
print(a)

[[ 0.90841688 -1.1615594   1.00485053  0.66247511  0.08546335]
 [ 1.80188667 -0.11353778  1.20319182  0.60443705 -0.60773928]]

[[ 0.90841688 -1.1615594   1.00485053  0.66247511  0.08546335]
 [ 1.80188667 -0.11353778  1.20319182  0.60443705 -0.60773928]]


In [36]:
# Numpyの配列ようにはsavaとload，あるいはsavetextとloadtextがある．
a = np.array([1,2,3])

# 保存
np.save("a.npy", a) # バイナリで保存
np.savetxt("a.data", a) # テキストで保存

# 読み出し
b1 = np.load("a.npy")
b2 = np.loadtxt("a.data")

print(b1,"\t", b2)

[1 2 3] 	 [ 1.  2.  3.]


## Neural Networkのおさらい  
- 基本的にはm次元のベクトル**x**をn次元のベクトル**y**に写す関数fを推定する学習方法．パラメータ**θ**を推定する．
- 関数を推定するため回帰の問題が対象だが，softmaxを利用することで分類問題へも応用可能．
- オンライン学習：データごとにパラメータを更新．データセットが大きい時に有効
- バッチ学習：データ全体を使ってパラメータを更新．データセットが小さいときに有効．
- ミニバッチ学習：オンライン学習とバッチ学習の中間．勾配が安定する効果があると言われている．

## Chainerの使い方
- Chainerの仕組みは合成関数を計算グラフで表現すると理解しやすい
- 計算グラフ：変数を表す○ノード，関数を表す⬜︎ノード，それらを向きのあるエッジで結んだグラフ
- Chainerではまず計算グラフで順方向に計算を行い，各ノードにその結果などの情報を保持させておき，次に逆向きにたどることで微分値を得るので，一度は順方向に計算しないといけないことに注意．
- **逆向きにたどる際に複数のパスがある場合は，パスごとの和になる**
$$ \frac{∂z}{∂x} = \frac{∂z}{∂y1} \frac{∂y1}{∂x} + \frac{∂z}{∂y2} \frac{∂y2}{∂x}  $$


In [39]:
# Chainerでオブジェクトを利用するにはモジュールのインポートが必要． 
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable, optimizers, serializers, utils # serializer：並直列変換回路
from chainer import Link, Chain, ChainList
import chainer.functions as F  # s!!
import chainer.links as L

In [37]:
# Variable：Chainerでは実数のタイプはnp.float32, 整数のタイプはnp.int32に固定しておく必要がある．生成時に指定してもいいが，型を変換するastypeを使うほうが応用が効く
x1 = Variable(np.array([1],  dtype=np.float32)) # dtypeの位置に注意．
x2 = Variable(np.array([2]).astype(np.float32)) # astypeの位置に注意
x3 = Variable(np.array([3]).astype(np.float32)) # astypeの位置に注意

z = (x1 - 2*x2 - 1)**2 + (x2 * x3 -1)**2 + 1
print("z:", z.data) # data属性で変数の中身を参照できる．

# 逆向きの計算を行うことで微分値を得られる．
z.backward()
print("x1_grad: ", x1.grad)
print("x2_grad: ", x2.grad)
print("x3_grad: ", x3.grad)

z: [ 42.]
x1_grad:  [-8.]
x2_grad:  [ 46.]
x3_grad:  [ 20.]


In [56]:
# functions：variableを変数としてもつ関数はfunctionsパッケージの中で提供されている．
x = Variable(np.array([-0.5], dtype=np.float32))
print("sigmoid(x): ", F.sigmoid(x).data)

z = F.cos(x)
print("cos(x): ", z.data)

# 微分
z.backward()
print("x_grad: ", x.grad)
print("-sin(x): ", -1.0 * F.sin(x).data)
# 変数が多次元である場合は，関数の傾きの次元をあらかじめ教えておく必要がある

sigmoid(x):  [ 0.37754068]
cos(x):  [ 0.87758255]
x_grad:  [ 0.47942555]
-sin(x):  [ 0.47942555]


In [73]:
# links：パラメータがある関数． Chainerで行えることは， links内におけるパラメータを推定すること．　自分が考えたモデル(合成関数)がlinks内の関すやfunctions内の関数を単純に組み合わせて表現できればほぼ完成．
# *functions内の関数にはパラメータがない
h = L.Linear(3, 4) # 線形作用素． y = Wx + b． 引数 = (入力層の次元, 出力層の次元)． linksの代表的な関数． パラメータはW(初期値:適当な数)とb(初期値:0)
print("W \n", h.W.data, end="\n\n") 
print("b \n", h.b.data, end="\n\n")

# 入力はバッチ(データの集合)で与えなければならない
x = Variable(np.array(np.arange(6)).astype(np.float32).reshape(2, 3))
y = h(x)
print("y \n", y.data, end="\n\n")

# 確認
w = h.W.data
x0 = x.data
print("y_ \n ", x0.dot(w.T) + h.b.data)

W 
 [[ 0.21137963 -0.5108822   0.71134382]
 [-0.10070859 -0.06428391  0.16123502]
 [ 0.08770657  0.52325684 -1.49643755]
 [ 0.68550384 -0.30790746  0.26451978]]

b 
 [ 0.  0.  0.  0.]

y 
 [[ 0.91180545  0.25818613 -2.46961832  0.2211321 ]
 [ 2.14732909  0.24691373 -5.12604046  2.14748049]]

y_ 
  [[ 0.91180545  0.25818613 -2.46961832  0.2211321 ]
 [ 2.14732909  0.24691373 -5.12604046  2.14748049]]


In [ ]:
# Chainクラス：モデルを定義．自身の定義するモデルは， Chainクラスを継承したクラス(例：MyChain)を定義することで行う．
class MyChain(Chain):
    # 合成関数内のlinks内の関数を列挙． 順番は無関係で， 全て使う必要もない．
    def __init__(self):
        super(MyChain, self).__init__(
            l1 = L.Linear(4,3), 
            l2 = L.Linear(3,3),
        )
        
    # 損失関数． __call__の部分に順方向の計算を書くほうがスマートだが， 多少変でも，「損失関数を書く」と決めておいたほうがプログラムは簡単になる．
    def __call__(self, x, y):
        fv = self.fwd(x, y)
        loss = F.mean_squared_error(fv, y)
        return loss
    
    # 順方向の計算
    def fwd(x, y):
        return F.sigmoid(self.l1(x))
        
# 最適化のための初期設定
model = MyChain() # モデルを生成
optimizer = optimizer.SGD()# 最適化のアルゴリズムの選択． Adamは比較的高速に良い値を出すため， 通常はAdamを使う．
optimizer.setup(model) # アルゴリズムにモデルをセット

# 1つの訓練データのバッチ(x,y)を与えると， パラメータが１回更新される
model.zerografs() # 勾配の初期化
loss = model(x,y) # 順方向に計算して誤差を算出
loss.backward() # 逆方向の計算，勾配の計算
optimizer.update() # パラメータを更新 

## Chainerの利用例　


【Chainerのプログラムの雛形】 *下の例の方がサンプルコードとして使える．  

    # Step1. データの準備．ここが面倒な場合が多い  

    # Step2. モデルを記述． __init__ と __call__ の部分は必須．   
    class MyModel(Chain): 
        def __init__(self):
            super(MyModel, self).__init__(
                # パラメータを含む関数(link)の宣言．順番は無関係で， 全て使う必要もない．カンマを忘れないように．
            )
        
        def __call__(self, x, y):
            # 損失関数． 順方向の計算は別のメソッドfwd(x)に分けると見通しがよくなる．

        def fwd(self, x):
            # 順伝播

    # Step3. モデルと最適化アルゴリズムの設定． ほぼお約束の3行． とりあえずAdamでいい  
    model = MyModel() # モデルの生成
    optimizer = optimizers.Adam() # 最適化アルゴリズムの選択
    optimizer.setup(model) # アルゴリズムにモデルをセット

    # Step4. 学習． かなり時間がかかる． ほぼお約束の4行．  
    for epoch in range(1000): # 1000：繰り返し回数．
        # データの加工
        x = Variable(xtrain.astype(np.float32))
        y = Variable(ytrain.astype(np.float32))
        model.zerograds() # 勾配初期化
        loss = model(x, y) # 誤差計算
        loss.backward() # 勾配計算
        optimizer.update() # パラメータ更新

    # Step.5 結果の出力

In [223]:
# Iris：アヤメの3クラス分類問題
# データの読み込み
from sklearn import datasets # 機械学習のプログラムを作る際はscikit-learnをインストールしておくと色々便利． 
iris = datasets.load_iris()
X = iris.data.astype(np.float32) # (150, 4)
Y = iris.target.astype(np.int32) # (150, )
N = Y.size # 150

# one-hot表現
Y2 = np.zeros([N, 3])
for i in range(N):
    Y2[i, Y[i]] = 1.0

# 訓練データとテストデータに分割
index = np.arange(N)
xtrain = X[index[index%2 != 0], :] # 奇数番目を訓練データに
ytrain = Y2[index[index%2 != 0], :]
xtest = X[index[index%2 == 0], :] # 偶数番目をテストデータに
yans = Y[index[index%2 == 0]]

# データの確認(偏ってる)
print(Y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


### Sample


In [226]:
# Step1. データの準備．ここが面倒な場合が多い  

# Step2. モデルを記述． __init__ と __call__ の部分は必須．   
class IrisChain(Chain): 
    def __init__(self):
        # パラメータを含む関数(link)の宣言．順番は無関係で， 全て使う必要もない．カンマを忘れないように．
        super(IrisChain, self).__init__(
            l1 = L.Linear(4, 6), 
            l2 = L.Linear(6, 3), 
        )

    # 損失関数． 順方向の計算は別のメソッドfwd(x)に分けると見通しがよくなる．
    def __call__(self, x, y):
        return F.mean_squared_error(self.fwd(x), y) # softmax_cross_entropyを使用する場合，教師ラベルは整数

    # 順伝播
    def fwd(self, x):
        h1 = F.sigmoid(self.l1(x))
        h2 = self.l2(h1) # 出力層には活性化関数を適応していない
        return h2

# Step3. モデルと最適化アルゴリズムの設定． ほぼお約束の3行． とりあえずAdamでいい  
model = IrisChain() # モデルの生成
optimizer = optimizers.Adam() # 最適化アルゴリズムの選択
optimizer.setup(model) # アルゴリズムにモデルをセット

# Step4. 学習． かなり時間がかかる． ほぼお約束の4行．  
for epoch in range(1000): # 1000：繰り返し回数．
    # データの加工
    ##  バッチ処理
    x = Variable(xtrain.astype(np.float32)) 
    y = Variable(ytrain.astype(np.float32))
    
    ## ミニバッチ
#     n = 75 # データのサイズ
#     bs = 24 # バッチサイズ
#     for j in range(5000): # データ全体の学習回数
#         accum_loss = None
#         sffindex = np.random.permutation(n)
#         for i in range(0, n, bs):
#             x = Variable(xtrain[sffindex[i:(i+bs) if (i+bs) < n else n]].astype(np.float32))
#             y = Variable(ytrain[sffindex[i:(i+bs) if (i+bs) < n else n]].astype(np.float32))
    
    model.zerograds() # 勾配初期化
    loss = model(x, y) # 誤差計算
    loss.backward() # 勾配計算
    optimizer.update() # パラメータ更新

# Step5. 結果の出力
xt = Variable(xtest) # volatile = "on"：テストでは計算グラフを使って勾配を求める必要がないため
yt = model.fwd(xt)
ans = yt.data
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    _cls = np.argmax(ans[i, :])
    if _cls == yans[i]:
        ok += 1

print(ok, " / ", nrow, " = ", (ok * 1.0) / nrow)

69  /  75  =  0.92
